<a href="https://colab.research.google.com/github/acastillo5248/CKY-Parser/blob/master/FSDS_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

# LSTM Test

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Load Train Data
def load_train_data():
    data = yf.download('XOP', start='2022-4-1', end='2024-3-1', interval='1d')
    data['MA_5'] = data['Close'].rolling(window=5).mean()
    data['MA_10'] = data['Close'].rolling(window=10).mean()
    #data['Volatility'] = (data['High'] - data['Low']) / data['Open']
    data['Momentum'] = data['Close'].pct_change(periods=5)
    #data['Daily_Range'] = data['High'] - data['Low']
    data['Target'] = data['Close'].shift(-1)  # Predict next day's closing price
    data = data.dropna()  # Remove NaN rows
    return data[['Open', 'Close', 'MA_5', 'MA_10' , 'Momentum', 'Target']]

# Preprocess Data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)  # This returns a NumPy array
    return np.array(scaled_data), scaler

# Create Sequences for Training
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, :-1])  # Use all but the last column as input
        y.append(data[i + seq_length, -1])    # Predict the last column (Target)
    return np.array(X), np.array(y)

# Load and preprocess training data
train_data = load_train_data()
scaled_train_data, scaler = preprocess_data(train_data)

# Generate training sequences
X_train, y_train = create_sequences(scaled_train_data, seq_length=30)

# Reshape input for LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))

# Build LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Predict closing price
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Load Test Data (Daily and Hourly)
def load_test_data():
    hourly_data = yf.download('XOP', start='2024-3-2', end='2024-12-1', interval='1h')
    daily_data = yf.download('XOP', start='2024-3-2', end='2024-12-1', interval='1d')
    return daily_data[['Open', 'Close']], hourly_data[['Open', 'Close']]

daily_test_data, hourly_test_data = load_test_data()
print(daily_test_data.head())
print(hourly_test_data.head())
# Preprocess hourly data for predictions
scaled_daily_data, _ = preprocess_data(daily_test_data)
X_test, y_test = create_sequences(scaled_daily_data, seq_length=30)

# Reshape test data for LSTM
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

# Generate predictions
predictions = model.predict(X_test)

# Reverse scaling for predictions and actual values
predicted_prices = np.zeros((predictions.shape[0], scaled_train_data.shape[1]))  # Placeholder
predicted_prices[:, -1] = predictions[:, 0]  # Fill in predictions
predicted_prices = scaler.inverse_transform(predicted_prices)[:, -1]  # Extract target column

actual_prices = np.zeros((y_test.shape[0], scaled_train_data.shape[1]))  # Placeholder
actual_prices[:, -1] = y_test  # Fill in actual target values
actual_prices = scaler.inverse_transform(actual_prices)[:, -1]

def trading_strategy(daily_test_data, hourly_test_data, predicted_prices, actual_prices, alpha=0.02, beta=0.05, stop_loss=0.1, take_profit=0.1):
    total_profit_loss = 0
    daily_results = []

    # Align lengths: add NaNs to hourly data for the first `seq_length`
    seq_length = 30
    idx = 0
    for date, row in daily_test_data.iterrows():
        if date not in daily_test_data.index:
            continue  # Skip dates not present in daily_test_data
        print(date)
        daily_open = row['Open'][0]
        daily_close = row['Close'][0]

        # Initialize daily profit/loss
        daily_profit_loss = 0
        shares_to_buy = 0
        if idx >= len(predicted_prices):
          break
        predicted_price = predicted_prices[idx]
        actual_price = actual_prices[idx]
        # Decision logic
        if predicted_price > actual_price * (1 + beta):  # Strong buy signal
            shares_to_buy = 8
        elif predicted_price > actual_price * (1 + 0.01):  # Weak buy signal
            shares_to_buy = 3
        elif predicted_price < actual_price * (1 - alpha):  # Expected price fall
            shares_to_buy = 0

        # Process hourly data for the day
        for time, hour_row in hourly_test_data.iterrows():
            hour_price = hour_row['Close'][0]

            if shares_to_buy == 0:
                break
            elif 1 - hour_price/daily_open < stop_loss:
                daily_profit_loss = (hour_price-daily_open)*shares_to_buy
                break
            elif 1 - daily_open/hour_price < take_profit:
                daily_profit_loss = (hour_price-daily_open)*shares_to_buy
                break
            else:
                daily_profit_loss = (hour_price-daily_open)*shares_to_buy

        total_profit_loss += daily_profit_loss
        strategy = "Buy" if shares_to_buy > 0 else "Hold"
        daily_results.append((date, strategy, shares_to_buy, daily_profit_loss))
        idx += 1
    return daily_results, total_profit_loss




# Run the strategy
daily_results, total_profit_loss = trading_strategy(daily_test_data, hourly_test_data, predicted_prices, actual_prices)

# Display results
print("Date       | Strategy | Shares | Daily Profit/Loss")
print("-------------------------------------------------")
for result in daily_results:
    date, strategy, shares, profit_loss = result
    print(f"{date} | {strategy:<8} | {shares:<6} | {profit_loss:.2f}")

print(f"\nTotal Profit/Loss: {total_profit_loss:.2f}")


[*********************100%***********************]  1 of 1 completed

Epoch 1/20



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0925 - val_loss: 0.0040
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0208 - val_loss: 0.0042
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0177 - val_loss: 0.0032
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0162 - val_loss: 0.0051
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0128 - val_loss: 0.0033
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0142 - val_loss: 0.0032
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0120 - val_loss: 0.0040
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0104 - val_loss: 0.0038
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0121 - val_loss: 0.0028
Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0125 - val_loss: 0.0025
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0110 - val_loss: 0.0028
Epoch 12/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0106 - val_loss: 0.0

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price             Open       Close
Ticker             XOP         XOP
Date                              
2024-03-04  142.839996  140.830002
2024-03-05  140.250000  141.419998
2024-03-06  143.039993  141.440002
2024-03-07  141.860001  142.350006
2024-03-08  142.729996  143.100006
Price                            Open       Close
Ticker                            XOP         XOP
Datetime                                         
2024-03-04 14:30:00+00:00  142.839996  142.539993
2024-03-04 15:30:00+00:00  142.490005  142.270004
2024-03-04 16:30:00+00:00  142.250000  141.699997
2024-03-04 17:30:00+00:00  141.690002  142.130005
2024-03-04 18:30:00+00:00  142.119995  141.809998


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2024-03-04 00:00:00
2024-03-05 00:00:00
2024-03-06 00:00:00
2024-03-07 00:00:00
2024-03-08 00:00:00
2024-03-11 00:00:00
2024-03-12 00:00:00
2024-03-13 00:00:00
2024-03-14 00:00:00
2024-03-15 00:00:00
2024-03-18 00:00:00
2024-03-19 00:00:00
2024-03-20 00:00:00
2024-03-21 00:00:00
2024-03-22 00:00:00
2024-03-25 00:00:00
2024-03-26 00:00:00
2024-03-27 00:00:00
2024-03-28 00:00:00
2024-04-01 00:00:00
2024-04-02 00:00:00
2024-04-03 00:00:00
2024-04-04 00:00:00
2024-04-05 00:00:00
2024-04-08 00:00:00
2024-04-09 00:00:00
2024-04-10 00:00:00
2024-04-11 00:00:00
2024-04-12 00:00:00
2024-04-15 00:00:00
2024-04-16 00:00:00
2024-04-17 00:00:00
2024-04-18 00:00:00
2024-04-19 00:00:00
2024-04-22 00:00:00
2024-04-23 00:00:00
2024-04-24 00:00:00
2024-04-25 00:00:00
2024-04-26 00:00:00
2024-04-29 00:00:00
2024-04-30 00:00:00
2024-05-01 00:00:00
2024-05-02 00:00:00
2024-05-03 00:00:00
2024-05-06 00:00:00
2024-05-07 00:00:00
2024-05-08 00:00:00
2024-05-09 00:00:00
20

<ipython-input-25-f045e5047979>:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  daily_open = row['Open'][0]
<ipython-input-25-f045e5047979>:94: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  daily_close = row['Close'][0]
<ipython-input-25-f045e5047979>:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hour_price = hour_row['Close'][0]


2024-07-31 00:00:00 | Buy      | 3      | -15.63
2024-08-01 00:00:00 | Hold     | 0      | 0.00
2024-08-02 00:00:00 | Hold     | 0      | 0.00
2024-08-05 00:00:00 | Hold     | 0      | 0.00
2024-08-06 00:00:00 | Hold     | 0      | 0.00
2024-08-07 00:00:00 | Hold     | 0      | 0.00
2024-08-08 00:00:00 | Hold     | 0      | 0.00
2024-08-09 00:00:00 | Hold     | 0      | 0.00
2024-08-12 00:00:00 | Hold     | 0      | 0.00
2024-08-13 00:00:00 | Hold     | 0      | 0.00
2024-08-14 00:00:00 | Buy      | 3      | 15.99
2024-08-15 00:00:00 | Hold     | 0      | 0.00
2024-08-16 00:00:00 | Hold     | 0      | 0.00
2024-08-19 00:00:00 | Hold     | 0      | 0.00
2024-08-20 00:00:00 | Hold     | 0      | 0.00
2024-08-21 00:00:00 | Hold     | 0      | 0.00
2024-08-22 00:00:00 | Hold     | 0      | 0.00
2024-08-23 00:00:00 | Hold     | 0      | 0.00
2024-08-26 00:00:00 | Hold     | 0      | 0.00
2024-08-27 00:00:00 | Hold     | 0      | 0.00
2024-08-28 00:00:00 | Hold     | 0      | 0.00
2024-08-29